In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
#import sys
#sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))


Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Python/JHU_553.640/

/content/drive/My Drive/Python/JHU_553.640


In [3]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
from datetime import datetime

# multiple output in notebook without print()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Other options
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)


### Data preprocessing- Smoothed yield curve data

In [4]:
# Data loading
# Using smoothed US Treasury yield curve data from Fed
# https://www.federalreserve.gov/econres/feds/the-us-treasury-yield-curve-1961-to-the-present.htm
df = pd.read_csv('feds200628.csv')

# Convert the Date column to yyyy-mm-dd format
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

# Leave only zero-coupon yield data
maturities = ['SVENY' + str(i).zfill(2) for i in range(1, 31)]
columns_to_keep = ['Date'] + [col for col in maturities if col in df.columns]
df = df[columns_to_keep]

# Change columns' names
name = ['Date'] + [str(i) + 'Y' for i in range(1, 31)]
df.columns = name

# Fill NaN values with the previous non-NaN value (forward fill)
df = df.fillna(method='ffill')
# Drop NaN values
df = df.dropna()

# Set 'Date' column as index
df.set_index('Date', inplace= True)

# Convert the index to datetime type
df.index = pd.to_datetime(df.index)


### Data preprocessing- FRED data

In [5]:
# Define the start and end dates for fetching data
start_date = '1961-01-01'
today = datetime.today().date()
end_date = today

# Fetch 3M, 6M yield and NASDAQ data from FRED using pandas_datareader
symbols_d = {
    '3M': 'DGS3MO',
    '6M': 'DGS6MO',
    'NASDAQ': 'NASDAQCOM'
}

# Fetch industrial production and consumer sentiment data from FRED using pandas_datareader
symbols_m = {
    'IP': 'INDPRO',
    'CS': 'UMCSENT'
}

# Create empty dataframe
fred_d = pd.DataFrame()
fred_m = pd.DataFrame()

# Load data from FRED
for label, symbol in symbols_d.items():
    fred_d[label] = pdr.DataReader(symbol, 'fred', start_date, end_date)

for label, symbol in symbols_m.items():
    fred_m[label] = pdr.DataReader(symbol, 'fred', start_date, end_date)

# Fill NaN values with the previous non-NaN value (forward fill)
fred_d = fred_d.fillna(method = 'ffill')
fred_d = fred_d.dropna()

fred_m = fred_m.fillna(method = 'ffill')
fred_m.index = fred_m.index.strftime('%Y-%m')
fred_m = fred_m.dropna()

### Data merging and Create monthly data

In [8]:
# Data merging
df_daily = pd.merge(df, fred_d, how = 'left', left_index=True, right_index=True)

# Change the columns' order
new_columns = ['3M', '6M'] + df.columns.tolist() + ['NASDAQ']
df_daily = df_daily.reindex(columns = new_columns)

# Identify the stock market index column
stock_index_col = 'NASDAQ'  # Replace with your actual stock index column name

# Get all yield columns (all columns except the stock index column)
yield_columns = [col for col in df_daily.columns if col != stock_index_col]

# Create a new column with year-month format
df_daily['year_month'] = df_daily.index.strftime('%Y-%m')

# Create two separate groups:
# 1. For yield columns - take the last day of each month
df_monthly_yields = df_daily.groupby('year_month')[yield_columns].last()

# 2. For stock index column - take the average of each month
df_monthly_stock = df_daily.groupby('year_month')[stock_index_col].mean()

# Combine the two results
df_monthly = df_monthly_yields.copy()
df_monthly[stock_index_col] = df_monthly_stock

# If 'year_month' somehow ended up in the columns, drop it
if 'year_month' in df_monthly.columns:
    df_monthly = df_monthly.drop(columns=['year_month'])

if 'year_month' in df_daily.columns:
    df_daily = df_daily.drop(columns=['year_month'])

## Add monthly industrial production and consumer sentiment index to the monthly dataframe
df_monthly = pd.merge(df_monthly, fred_m, how = 'left', left_index=True, right_index=True)
df_monthly.dropna(inplace = True)

# Rename index
df_monthly.index.name = 'Date'

## Pre-processing for macro-finance variables
# Apply log transformation to NASDAQ
df_monthly['NASDAQ'] = np.log(df_monthly['NASDAQ'])

# Apply first difference to the log-transformed NASDAQ
# This will create NaN for the first row
df_monthly['NASDAQ'] = df_monthly['NASDAQ'].diff()

# Apply log transformation to IP and CS columns
df_monthly['IP'] = np.log(df_monthly['IP'])
df_monthly['CS'] = np.log(df_monthly['CS'])
df_monthly.dropna(inplace = True)

# Save the collapsed dataset
df_monthly.to_csv('df_monthly.csv', index=True)

df_monthly.head()
df_monthly.tail()



,3M,6M,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,11Y,12Y,13Y,14Y,15Y,16Y,17Y,18Y,19Y,20Y,21Y,22Y,23Y,24Y,25Y,26Y,27Y,28Y,29Y,30Y,NASDAQ,IP,CS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1985-12,7.28,7.44,7.6074,7.9363,8.1951,8.4044,8.5788,8.7280,8.8590,8.9762,9.0828,9.1810,9.2721,9.3571,9.4367,9.5114,9.5816,9.6475,9.7095,9.7677,9.8223,9.8737,9.9219,9.9671,10.0096,10.0495,10.0870,10.1222,10.1553,10.1865,10.2158,10.2435,0.025897,4.013772,4.542230
1986-01,7.19,7.41,7.5967,7.9420,8.2217,8.4533,8.6486,8.8159,8.9613,9.0890,9.2020,9.3027,9.3930,9.4742,9.5475,9.6138,9.6741,9.7289,9.7789,9.8246,9.8664,9.9047,9.9400,9.9725,10.0025,10.0302,10.0560,10.0798,10.1020,10.1228,10.1421,10.1602,0.025059,4.018998,4.560173
1986-02,7.24,7.36,7.4333,7.6473,7.7883,7.8867,7.9608,8.0217,8.0756,8.1258,8.1740,8.2209,8.2666,8.3110,8.3539,8.3952,8.4346,8.4721,8.5075,8.5409,8.5723,8.6018,8.6293,8.6551,8.6792,8.7017,8.7228,8.7425,8.7609,8.7781,8.7943,8.8095,0.060538,4.012111,4.563306
1986-03,6.53,6.57,6.8290,7.0092,7.1372,7.2310,7.3023,7.3587,7.4050,7.4443,7.4786,7.5091,7.5367,7.5619,7.5850,7.6064,7.6261,7.6443,7.6611,7.6768,7.6912,7.7046,7.7170,7.7286,7.7393,7.7492,7.7585,7.7671,7.7752,7.7827,7.7898,7.7964,0.054705,4.005018,4.554929
1986-04,6.28,6.44,6.6112,6.9315,7.1297,7.2558,7.3405,7.4018,7.4505,7.4923,7.5306,7.5669,7.6018,7.6355,7.6680,7.6993,7.7291,7.7573,7.7841,7.8092,7.8328,7.8549,7.8755,7.8947,7.9127,7.9294,7.9450,7.9596,7.9732,7.9859,7.9978,8.0090,0.037468,4.006151,4.566429


,3M,6M,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,11Y,12Y,13Y,14Y,15Y,16Y,17Y,18Y,19Y,20Y,21Y,22Y,23Y,24Y,25Y,26Y,27Y,28Y,29Y,30Y,NASDAQ,IP,CS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-11,4.58,4.42,4.3134,4.1558,4.0698,4.0323,4.0273,4.0435,4.0730,4.1103,4.1514,4.1937,4.2353,4.2750,4.3119,4.3457,4.3759,4.4025,4.4255,4.4448,4.4606,4.4730,4.4823,4.4885,4.4919,4.4926,4.4909,4.4869,4.4808,4.4728,4.4630,4.4516,0.034873,4.624599,4.273884
2024-12,4.37,4.24,4.1982,4.2273,4.2638,4.3058,4.3514,4.3994,4.4485,4.4975,4.5458,4.5924,4.6370,4.6789,4.7178,4.7534,4.7855,4.8140,4.8387,4.8596,4.8768,4.8901,4.8998,4.9058,4.9083,4.9074,4.9032,4.8959,4.8855,4.8723,4.8565,4.8380,0.040400,4.635871,4.304065
2025-01,4.31,4.28,4.2066,4.1969,4.2342,4.2866,4.3430,4.3992,4.4533,4.5046,4.5527,4.5976,4.6392,4.6777,4.7131,4.7456,4.7754,4.8025,4.8271,4.8494,4.8695,4.8875,4.9036,4.9178,4.9303,4.9412,4.9506,4.9586,4.9653,4.9708,4.9751,4.9783,-0.009922,4.638042,4.272491
2025-02,4.32,4.25,4.0849,3.9687,3.9508,3.9742,4.0152,4.0635,4.1140,4.1643,4.2130,4.2594,4.3032,4.3443,4.3826,4.4181,4.4508,4.4809,4.5085,4.5336,4.5564,4.5770,4.5955,4.6120,4.6267,4.6396,4.6509,4.6606,4.6688,4.6757,4.6813,4.6857,0.000929,4.646506,4.169761
2025-03,4.32,4.23,4.0087,3.8759,3.8533,3.8822,3.9351,3.9983,4.0650,4.1315,4.1959,4.2571,4.3146,4.3681,4.4176,4.4631,4.5046,4.5424,4.5766,4.6073,4.6348,4.6592,4.6807,4.6994,4.7156,4.7293,4.7409,4.7503,4.7578,4.7634,4.7673,4.7697,-0.093290,4.643325,4.043051
